In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import pandavro as pdx
from shl.prepare import fill_na, calculate_abs_values, calculate_change, calculate_pct_change, calculate_window, calculate_shift
from typing import List, Optional, Tuple

os.chdir('../')
data_path = './data'

pd.set_option('display.max_columns', None)

In [82]:
def prepare_features(df, df_label, window_sizes, window_center, window_functions, shift_periods, shift_column_patterns):  
        
    # Add time rounded to seconds
    df['epoch_time'] = df['epoch_time'].round(-3)
    
    # Group values by rounded time    
    df = df[['epoch_time', 'cell_type', 'asuLevel']].groupby(['epoch_time', 'cell_type'], as_index=False).median()    
   
    
    # Join with label
    if df_label is not None:
        df = df_label.merge(df[df['cell_type'] == 'LTE'][['epoch_time', 'asuLevel']].rename(columns={'asuLevel': 'asuLevel_LTE'}), on='epoch_time', how='left') \
                     .merge(df[df['cell_type'] == 'GSM'][['epoch_time', 'asuLevel']].rename(columns={'asuLevel': 'asuLevel_GSM'}), on='epoch_time', how='left') \
                     .merge(df[df['cell_type'] == 'WCDMA'][['epoch_time', 'asuLevel']].rename(columns={'asuLevel': 'asuLevel_WCDMA'}), on='epoch_time', how='left') \
                     .drop('label', axis=1)       
    
    # Ensure that values are sorted before calculating diff
    df = df.sort_values(by='epoch_time')    
    
    # Fill NA 
    df = fill_na(df)
    
    # Calculate percent change
    df_pct = calculate_pct_change(df)    
    df_pct.drop(['epoch_time_pct_change'], axis=1, inplace=True) 
    fill_na(df_pct)
    
    # Merge new features back to the main dataframe
    df = df[['epoch_time']].merge(df_pct, left_index=True, right_index=True, how='left')
    
    # Add moving windows features
    df = calculate_window(df, columns=None, functions=window_functions, window_sizes=window_sizes, window_center=True)       
    
    # Add shifted features
    calculate_shift(df, periods=shift_periods, columns_patterns=shift_column_patterns) 
   
    return df

In [83]:
# Run processing

settings = {
'window_sizes': [5, 10, 30, 60, 300],
'window_center': True,
'window_functions': ['mean', 'std'],
'shift_periods': [5, 10, 30, 60, 300],
'shift_column_patterns': ['window_5_', 'window_10_', 'window_30_', 'window_60_', 'window_300_'],
}

input_file_name = 'Cells.parquet'
input_file_name_label = 'Label.parquet'
data_types = ['validate', 'train', 'test']
#data_types = ['validate']
full_input_file_names = [os.path.join(data_path, data_type, input_file_name) for data_type in data_types]
full_input_file_names_label = [os.path.join(data_path, data_type, input_file_name_label) for data_type in data_types]

base_output_file_name = 'features_cells'
output_format = 'parquet'
output_file_name = base_output_file_name + '.' + output_format
full_output_file_names = [os.path.join(data_path, data_type, output_file_name) for data_type in data_types]

for file_in, file_in_label, file_out in zip(full_input_file_names, full_input_file_names_label, full_output_file_names):
    df = pd.read_parquet(file_in)
    df_label = None
    if os.path.exists(file_in_label):
        df_label = pd.read_parquet(file_in_label)
    df = prepare_features(df, df_label, **settings)
    if output_format == 'parquet':
        df.to_parquet(file_out, index=False)
    elif output_format == 'avro':
        pdx.to_avro(file_out, df)        


In [84]:
df

,epoch_time,asuLevel_LTE_pct_change,asuLevel_GSM_pct_change,asuLevel_WCDMA_pct_change,asuLevel_LTE_pct_change_window_5_mean,asuLevel_LTE_pct_change_window_5_std,asuLevel_GSM_pct_change_window_5_mean,asuLevel_GSM_pct_change_window_5_std,asuLevel_WCDMA_pct_change_window_5_mean,asuLevel_WCDMA_pct_change_window_5_std,asuLevel_LTE_pct_change_window_10_mean,asuLevel_LTE_pct_change_window_10_std,asuLevel_GSM_pct_change_window_10_mean,asuLevel_GSM_pct_change_window_10_std,asuLevel_WCDMA_pct_change_window_10_mean,asuLevel_WCDMA_pct_change_window_10_std,asuLevel_LTE_pct_change_window_30_mean,asuLevel_LTE_pct_change_window_30_std,asuLevel_GSM_pct_change_window_30_mean,asuLevel_GSM_pct_change_window_30_std,asuLevel_WCDMA_pct_change_window_30_mean,asuLevel_WCDMA_pct_change_window_30_std,asuLevel_LTE_pct_change_window_60_mean,asuLevel_LTE_pct_change_window_60_std,asuLevel_GSM_pct_change_window_60_mean,asuLevel_GSM_pct_change_window_60_std,asuLevel_WCDMA_pct_change_window_60_mean,asuLevel_WCDMA_pct_change_window_60_std,asuLevel_LTE_pct_change_window_300_mean,asuLevel_LTE_pct_change_window_300_std,asuLevel_GSM_pct_change_window_300_mean,asuLevel_GSM_pct_change_window_300_std,asuLevel_WCDMA_pct_change_window_300_mean,asuLevel_WCDMA_pct_change_window_300_std,asuLevel_LTE_pct_change_window_5_mean_shift_5_past,asuLevel_LTE_pct_change_window_5_std_shift_5_past,asuLevel_GSM_pct_change_window_5_mean_shift_5_past,asuLevel_GSM_pct_change_window_5_std_shift_5_past,asuLevel_WCDMA_pct_change_window_5_mean_shift_5_past,asuLevel_WCDMA_pct_change_window_5_std_shift_5_past,asuLevel_LTE_pct_change_window_5_mean_shift_5_future,asuLevel_LTE_pct_change_window_5_std_shift_5_future,asuLevel_GSM_pct_change_window_5_mean_shift_5_future,asuLevel_GSM_pct_change_window_5_std_shift_5_future,asuLevel_WCDMA_pct_change_window_5_mean_shift_5_future,asuLevel_WCDMA_pct_change_window_5_std_shift_5_future,asuLevel_LTE_pct_change_window_10_mean_shift_10_past,asuLevel_LTE_pct_change_window_10_std_shift_10_past,asuLevel_GSM_pct_change_window_10_mean_shift_10_past,asuLevel_GSM_pct_change_window_10_std_shift_10_past,asuLevel_WCDMA_pct_change_window_10_mean_shift_10_past,asuLevel_WCDMA_pct_change_window_10_std_shift_10_past,asuLevel_LTE_pct_change_window_10_mean_shift_10_future,asuLevel_LTE_pct_change_window_10_std_shift_10_future,asuLevel_GSM_pct_change_window_10_mean_shift_10_future,asuLevel_GSM_pct_change_window_10_std_shift_10_future,asuLevel_WCDMA_pct_change_window_10_mean_shift_10_future,asuLevel_WCDMA_pct_change_window_10_std_shift_10_future,asuLevel_LTE_pct_change_window_30_mean_shift_30_past,asuLevel_LTE_pct_change_window_30_std_shift_30_past,asuLevel_GSM_pct_change_window_30_mean_shift_30_past,asuLevel_GSM_pct_change_window_30_std_shift_30_past,asuLevel_WCDMA_pct_change_window_30_mean_shift_30_past,asuLevel_WCDMA_pct_change_window_30_std_shift_30_past,asuLevel_LTE_pct_change_window_30_mean_shift_30_future,asuLevel_LTE_pct_change_window_30_std_shift_30_future,asuLevel_GSM_pct_change_window_30_mean_shift_30_future,asuLevel_GSM_pct_change_window_30_std_shift_30_future,asuLevel_WCDMA_pct_change_window_30_mean_shift_30_future,asuLevel_WCDMA_pct_change_window_30_std_shift_30_future,asuLevel_LTE_pct_change_window_60_mean_shift_60_past,asuLevel_LTE_pct_change_window_60_std_shift_60_past,asuLevel_GSM_pct_change_window_60_mean_shift_60_past,asuLevel_GSM_pct_change_window_60_std_shift_60_past,asuLevel_WCDMA_pct_change_window_60_mean_shift_60_past,asuLevel_WCDMA_pct_change_window_60_std_shift_60_past,asuLevel_LTE_pct_change_window_60_mean_shift_60_future,asuLevel_LTE_pct_change_window_60_std_shift_60_future,asuLevel_GSM_pct_change_window_60_mean_shift_60_future,asuLevel_GSM_pct_change_window_60_std_shift_60_future,asuLevel_WCDMA_pct_change_window_60_mean_shift_60_future,asuLevel_WCDMA_pct_change_window_60_std_shift_60_future,asuLevel_LTE_pct_change_window_300_mean_shift_300_past,asuLevel_LTE_pct_change_window_300_std_shift_300_past,asuLevel_GSM_pct_change_window_300_mean_shift_300_past

In [85]:
df.isnull().sum().sum()

0

### Export for analysis

In [56]:
# Join labels for analysis
# features_file = './data/validate/Cells.parquet'
# features = pd.read_parquet(features_file)
# features['epoch_time_s'] = features['epoch_time'].round(-3)

# label_file = './data/validate/Label.parquet'
# label = pd.read_parquet(label_file)

# features = label.merge(features).sort_values(by='epoch_time')
# features.to_csv('./data/cells_analysis.csv')
# features

,epoch_time,num_entries,cell_type,isRegistered,ci,MCC,MNC,PCI,TAC,asuLevel,dBm,level,cid,lac,PSC,epoch_time_s
0,1497426495739,4,LTE,1,1.290515e+08,234,10,398.0,1.440000e+02,54,-86,4,NaN,NaN,NaN,1497426496000
1,1497426495739,4,LTE,0,2.147484e+09,234,10,398.0,2.147484e+09,45,-95,4,NaN,NaN,NaN,1497426496000
2,1497426495739,4,LTE,0,2.147484e+09,234,10,19.0,2.147484e+09,39,-101,3,NaN,NaN,NaN,1497426496000
3,1497426495739,4,LTE,0,2.147484e+09,234,10,20.0,2.147484e+09,34,-106,2,NaN,NaN,NaN,1497426496000
4,1497426495745,4,LTE,1,1.290515e+08,234,10,398.0,1.440000e+02,54,-86,4,NaN,NaN,NaN,1497426496000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333896,7497441587427,6,GSM,0,NaN,234,10,NaN,NaN,0,-113,0,2.147484e+09,2.147484e+09,NaN,7497441587000
333897,7497441587427,6,GSM,0,NaN,234,10,NaN,NaN,0,-113,0,2.147484e+09,2.147484e+09,NaN,7497441587000
333898,7497441587427,6,GSM,0,NaN,234,10,NaN,NaN,0,-113,0,2.147484e+09,2.147484e+09,NaN,7497441587000
333899,7497441587427,6,GSM,0,NaN,234,10,NaN,NaN,0,-113,0,2.147484e+09,2.147484e+09,NaN,7497441587000
